## Export data to SpaceTX format

First create the subclasses of `FetchedTile` and `TileFetcher` required and then export the data into SpaceTX

In [1]:
%load_ext autoreload
%autoreload 2

import functools
import os
from typing import Mapping, Tuple, Union

import click
import numpy as np
from skimage.io import imread
from slicedimage import ImageFormat

from starfish import Codebook
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, CoordinateValue, Features

import pdb

@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path) -> np.ndarray:
    return imread(file_path)

class StarMapTile(FetchedTile):

    def __init__(
            self,
            file_path: str,
            coordinates: Mapping[Union[str, Coordinates], CoordinateValue]
    ) -> None:
        self.file_path = file_path
        self._coordinates = coordinates

    @property
    def shape(self) -> Mapping[Axes, int]:
#        print(np.shape(self.tile_data()))
#         return {Axes.Y: 7962, Axes.X: 3356}
#         return {Axes.Y: 1000, Axes.X: 500}
#         return {Axes.Y: 1024, Axes.X: 1024}
        return {Axes.Y: 239, Axes.X: 239}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
        return self._coordinates

    @staticmethod
    def crop(img) -> np.ndarray:
#         crp = img[2000:6000, 1000:1800]
#         crp = img[3000:4000, 1300:1800]
#         crp = img[2950:3150, 1330:1730]
#         crp = img[1024:2048, 0:1024]
        crp = img[6540:6779, 2953:3192]
        return crp
    
    def tile_data(self) -> np.ndarray:
#        return self.crop(imread(self.file_path))
       return (imread(self.file_path))
    
    def __str__(self) -> str:
        return self.file_path
    
class StarMapTileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
#         self.num_z = 11
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round1_2x10_1_FusionStitcher_C{ch_label+1}_Z{zplane_padded}.tif"  # translate to 3d
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
class StarMapDAPITileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round1_2x10_1_FusionStitcher_C0_Z{zplane_padded}.tif"
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
def format_data(input_dir, output_dir):
    
    primary_image_dimensions: Mapping[Axes, int] = {
        Axes.ROUND: 1,
        Axes.CH: 4,
        Axes.ZPLANE: 11,
    }
    
    aux_image_dimensions: Mapping[str, Mapping[Union[str, Axes], int]] = {
        "nuclei": {
            Axes.ROUND: 1,
            Axes.CH: 1,
            Axes.ZPLANE: 11
        }
    }
    
#     pdb.set_trace()
    write_experiment_json(
        path=output_dir,
        fov_count=1,
        tile_format=ImageFormat.TIFF,
        primary_image_dimensions=primary_image_dimensions,
        aux_name_to_dimensions=aux_image_dimensions,
        primary_tile_fetcher=StarMapTileFetcher(input_dir),
        aux_tile_fetcher={"nuclei": StarMapDAPITileFetcher(input_dir)},
        dimension_order=(Axes.ROUND, Axes.CH, Axes.ZPLANE)
    )
    
    codebook = [
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 0, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "GFP"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 1, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "RFP"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 2, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "Cy5"
        },
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 3, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "iRFP"
        }
    ]
    Codebook.from_code_array(codebook).to_json("/home/nomi/Desktop/starfish/experiment/codebook.json")

In [1]:
%load_ext autoreload
%autoreload 2

import functools
import os
from typing import Mapping, Tuple, Union

import click
import numpy as np
from skimage.io import imread
from slicedimage import ImageFormat

from starfish import Codebook
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, CoordinateValue, Features

import pdb

@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path) -> np.ndarray:
    return imread(file_path)

class StarMapTile(FetchedTile):

    def __init__(
            self,
            file_path: str,
            coordinates: Mapping[Union[str, Coordinates], CoordinateValue]
    ) -> None:
        self.file_path = file_path
        self._coordinates = coordinates

    @property
    def shape(self) -> Mapping[Axes, int]:
        print(np.shape(self.tile_data()))
#         return {Axes.Y: 7962, Axes.X: 3356}
#         return {Axes.Y: 1024, Axes.X: 1024}
#         return {Axes.Y: 1000, Axes.X: 500}
        return {Axes.Y: 239, Axes.X: 239}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
        return self._coordinates

    @staticmethod
    def crop(img) -> np.ndarray:
#         crp = img[2000:6000, 1000:1800]
#         crp = img[3000:4000, 1300:1800]
#         crp = img[2950:3150, 1330:1730]
#        crp = img[1024:2048, 0:1024]
        crp = img[6540:6779, 2953:3192]
        return crp
    
    def tile_data(self) -> np.ndarray:
        return self.crop(imread(self.file_path))
#         return (imread(self.file_path))
    
    def __str__(self) -> str:
        return self.file_path
    
class StarMapTileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
#         self.num_z = 11
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round1_2x10_1_FusionStitcher_C3_Z{zplane_padded}.tif"  # translate to 3d
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
class StarMapDAPITileFetcher(TileFetcher):
    
    def __init__(self, input_dir: str) -> None:
        self.input_dir = input_dir
        
    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        if zplane_label < 10:
            zplane_padded = f"00{zplane_label}"
        elif zplane_label < 100:
            zplane_padded = f"0{zplane_label}"
        else:
            zplane_padded = str(zplane_label)
        basename = f"2019-06-29_Justus_section3_round1_2x10_1_FusionStitcher_C{ch_label}_Z{zplane_padded}.tif"
        file_path = os.path.join(self.input_dir, basename)
        coordinates = {
            Coordinates.X: (0.0, 0.0001),
            Coordinates.Y: (0.0, 0.0001),
            Coordinates.Z: (0.0, 0.0001),
        }
        return StarMapTile(file_path, coordinates)
    
def format_data(input_dir, output_dir):
    
    primary_image_dimensions: Mapping[Axes, int] = {
        Axes.ROUND: 1,
        Axes.CH: 1,
        Axes.ZPLANE: 11,
    }
    
    aux_image_dimensions: Mapping[str, Mapping[Union[str, Axes], int]] = {
        "nuclei": {
            Axes.ROUND: 1,
            Axes.CH: 1,
            Axes.ZPLANE: 11
        }
    }
    
#     pdb.set_trace()
    write_experiment_json(
        path=output_dir,
        fov_count=1,
        tile_format=ImageFormat.TIFF,
        primary_image_dimensions=primary_image_dimensions,
        aux_name_to_dimensions=aux_image_dimensions,
        primary_tile_fetcher=StarMapTileFetcher(input_dir),
        aux_tile_fetcher={"nuclei": StarMapDAPITileFetcher(input_dir)},
        dimension_order=(Axes.ROUND, Axes.CH, Axes.ZPLANE)
    )
    codebook = [
        {
            Features.CODEWORD: [
                {Axes.ROUND.value: 0, Axes.CH.value: 0, Features.CODE_VALUE: 1}
            ],
            Features.TARGET: "RFP"
        }
    ]
    Codebook.from_code_array(codebook).to_json("/home/nomi/Desktop/starfish/experiment/codebook.json")

In [2]:
format_data("/home/nomi/Desktop/starfish/raw_data/2019-06-29_Justus_section3_round1_2x10_1",
           "/home/nomi/Desktop/starfish/experiment")

(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)


/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7f91770eb8f0> is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7f9176bf8b30> is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7f9176d860b0> is a low contrast image
  warn('%s is a low contrast image' % fname)


(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)
(239, 239)


/home/nomi/Desktop/starfish/.venv/lib/python3.7/site-packages/skimage/io/_io.py:141: UserWarning: <_io.BytesIO object at 0x7f91770ebad0> is a low contrast image
  warn('%s is a low contrast image' % fname)


## Load the experiment and visualize the codebook
Possibly necessary to copy the codebook information from `codebook_backup.json`.

In [2]:
%load_ext autoreload
%autoreload 2
from starfish import Experiment
from six.moves import urllib
from slicedimage.backends import CachingBackend, DiskBackend, HttpBackend, SIZE_LIMIT

baseurl="/home/nomi/Desktop/starfish/experiment/experiment.json"
experiment = Experiment.from_json(baseurl)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
experiment.codebook

<xarray.Codebook (target: 1, c: 1, r: 1)>
array([[[1]]], dtype=uint8)
Coordinates:
  * target   (target) object 'RFP'
  * c        (c) int64 0
  * r        (r) int64 0

## Visualize the FOV

In [4]:
fov = experiment["fov_000"]

In [5]:
image = fov.get_image("primary")

In [15]:
import starfish
%gui qt
ipython = get_ipython()
ipython.magic("gui qt5")
starfish.display(image.max_proj(Axes.ZPLANE))

100%|██████████| 1/1 [00:00<00:00, 232.35it/s]


## Project onto the Z axis
The slider in the viewer should now only have as many options as there are channels. Also clipping the image to remove background noise.

In [6]:
import starfish
from starfish.types import Axes
clipper = starfish.image.Filter.Clip(p_min=99)
glp = starfish.image.Filter.GaussianLowPass(
    sigma=(1, 1, 1),
    is_volume=True
)
z_proj: starfish.ImageStack = ((image.max_proj(Axes.ZPLANE)))

100%|██████████| 11/11 [00:00<00:00, 197.89it/s]


In [27]:
with open("/home/nomi/Desktop/pre-filter.svg", 'w') as file:
    print(f"{starfish.display(z_proj).to_svg()}", file=file)

## Apply Filters
Applying a bandpass and Gaussian low pass filters interleaved with clip filters.

In [7]:
from typing import Optional

from starfish import ImageStack
from starfish.image import Filter



def preprocess_fov(primary_fov_imagestack: ImageStack,
                  n_processes: Optional[int] = None,
                  is_volume: Optional[bool] = False) -> ImageStack:
   """Preprocess a Starfish field of view image stack in preparation for
   spot/pixel finding.

   NOTE: This preprocessing pipeline processes imagestacks in place!

   Args:
       primary_fov_imagestack (ImageStack): A starfish FOV Imagestack
       n_processes (Optional[int]): Number of processes to use for
           preprocessing steps. If None, uses the output of os.cpu_count().
           Defaults to None.

   Returns:
       ImageStack: A preprocessed starfish imagestack.
   """
   print("Applying First Clip...")
   first_clip = Filter.ClipPercentileToZero(p_min=75, p_max=100,
                                            is_volume=is_volume)
   first_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                  n_processes=n_processes)

   print("Applying Bandpass...")
   bpass = Filter.Bandpass(lshort=0.5, llong=7, threshold=1/(1<<16-1),
                           is_volume=is_volume)
   bpass.run(primary_fov_imagestack, in_place=True, verbose=True,
             n_processes=n_processes)

   print("Applying Second Clip...")
   second_clip = Filter.ClipValueToZero(v_min=1/(1<<16-1), is_volume=is_volume)
   second_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                   n_processes=n_processes)

   print("Applying Gaussian Low Pass...")
   z_gauss_filter = Filter.GaussianLowPass(sigma=(1, 0, 0), is_volume=True)
   z_gauss_filter.run(primary_fov_imagestack, in_place=True,
                      n_processes=n_processes)

   print("Applying Final Clips...")
   final_percent_clip = Filter.ClipPercentileToZero(p_min=90, min_coeff=1.75)
   final_percent_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                          n_processes=n_processes)

   final_value_clip = Filter.ClipValueToZero(v_max=1000/(1<<16-1))
   final_value_clip.run(primary_fov_imagestack, in_place=True, verbose=True,
                        n_processes=n_processes)

   return primary_fov_imagestack

In [8]:
z_proj = preprocess_fov(z_proj, n_processes=22)

100%|██████████| 1/1 [00:00<00:00, 130.04it/s]
0it [00:00, ?it/s]

Applying First Clip...


1it [00:00,  5.29it/s]
0it [00:00, ?it/s]

Applying Bandpass...


1it [00:00,  5.94it/s]
0it [00:00, ?it/s]

...

1it [00:00,  5.64it/s]


Applying Gaussian Low Pass...


0it [00:00, ?it/s]

Applying Final Clips...


1it [00:00,  5.61it/s]
1it [00:00,  5.69it/s]


In [30]:
with open("/home/nomi/Desktop/post-filter.svg", 'w') as file:
    print(f"{starfish.display(z_proj).to_svg()}", file=file)

Find spots
----------
Finally, a local blob detector that finds spots in each (z, y, x) volume
separately is applied. The user selects an "anchor round" and spots found in
all channels of that round are used to seed a local search across other rounds
and channels. The closest spot is selected, and any spots outside the search
radius (here 10 pixels) is discarded.

The Spot finder returns an IntensityTable containing all spots from round
zero. Note that many of the spots do _not_ identify spots in other rounds and
channels and will therefore fail decoding. Because of the stringency built
into the STARmap codebook, it is OK to be relatively permissive with the spot
finding parameters for this assay.

In [19]:
import starfish
import time
import numpy as np
lsbd = starfish.spots.DetectSpots.BlobDetector(
     min_sigma=.45,
     max_sigma=1.25,
     num_sigma=25,
     threshold=np.percentile(np.ravel(z_proj.xarray.values), 97),
     is_volume=False,
     overlap=0.7,
# #     exclude_border=2,
# #     anchor_round=0,
# #     search_radius=10,
 )
# tlmpf = starfish.spots.DetectSpots.TrackpyLocalMaxPeakFinder(
#     spot_diameter=1,  # must be odd integer
#     min_mass=0.02,|
#     max_size=5,  # this is max radius
#     separation=1,
#     noise_size=0.65,  # this is not used because preprocess is False
#     preprocess=False,
#     percentile=10,  # this is irrelevant when min_mass, spot_diameter, and max_size are set properly
#     verbose=True,
#     is_volume=True,
# )
lmpf = starfish.spots.DetectSpots.LocalMaxPeakFinder(
min_distance=1,
stringency=0,
min_obj_area=6,
max_obj_area=600,
verbose=True,
is_volume=False)

# intensities = lsbd.run(z_proj, n_processes=22)
# intensities = tlmpf.run(image, n_processes=22)
start: float = time.perf_counter()
intensities = lmpf.run(z_proj, n_processes=22)
print(f"Minutes elapsed: {(time.perf_counter())/60 - start}")

  0%|          | 0/100 [00:00<?, ?it/s]

Determining optimal threshold ...


 70%|███████   | 70/100 [00:00<00:00, 336.05it/s]

Stopping early at threshold=0.012798946576588081. Number of spots fell below: 3


computing final spots ...
Minutes elapsed: -1950433.720654375


Decode spots
------------
Next, spots are decoded.

In [20]:
decoded = experiment.codebook.decode_per_round_max(intensities.fillna(0))
decode_mask = decoded['target'] != 'nan'

In [ ]:
import sklearn
print(sorted(sklearn.neighbors.VALID_METRICS['ball_tree']))
decoded = experiment.codebook.decode_metric(intensities.fillna(0), 1, 0, 2, 'l2')
decode_mask = decoded['target'] != 'nan'

In [21]:
import pandas as pd
from starfish.types import Features
from starfish.core.types import DecodedSpots

df = pd.DataFrame(dict(decoded['features'].coords))
pixel_coordinates = pd.Index(['x', 'y', 'z'])
ds = DecodedSpots(df)

In [22]:
ds.data

,radius,x,y,z,features,xc,yc,zc,target,distance,passes_thresholds
0,1,134,229,0,0,0.000056,0.000096,0.00005,RFP,0.0,True
1,1,129,228,0,1,0.000054,0.000096,0.00005,RFP,0.0,True
2,1,196,218,0,2,0.000082,0.000092,0.00005,RFP,0.0,True
3,1,62,197,0,3,0.000026,0.000083,0.00005,RFP,0.0,True
4,1,60,190,0,4,0.000025,0.000080,0.00005,RFP,0.0,True
5,1,126,187,0,5,0.000053,0.000079,0.00005,RFP,0.0,True
6,1,170,186,0,6,0.000071,0.000078,0.00005,RFP,0.0,True
7,1,101,183,0,7,0.000042,0.000077,0.00005,RFP,0.0,True
8,1,98,180,0,8,0.000041,0.000076,0.00005,RFP,0.0,True
9,1,70,177,0,9,0.000029,0.000074,0.00005,RFP,0.0,True


In [23]:
%gui qt
with open("/home/nomi/Desktop/with-spots.svg", 'w') as file:
    print(starfish.display(
    z_proj, decoded[decode_mask], radius_multiplier=2, mask_intensities=0.0
    ).to_svg(), file=file)

In [43]:
counter={"GFP": 0, "RFP": 0, "Cy5": 0, "iRFP": 0}
for i,j in ds.data.iterrows():
    counter[j['target']]+=1
print(counter)

{'GFP': 0, 'RFP': 23, 'Cy5': 0, 'iRFP': 0}
